In [9]:
import cv2
import pytesseract
import os
import csv
import re

In [2]:
# get video
video_path = 'Ex1.mp4'
cap = cv2.VideoCapture(video_path)

In [3]:
# check whether the video is open
if not cap.isOpened():
    print("Error opening video file")
    exit()

In [4]:
# get frame rate of the video
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Frame rate: {fps} fps")  # check frame rate

Frame rate: 31.2718797294295 fps


In [5]:
# open a dirs to save frames
output_folder = 'frames_Ex_1'
os.makedirs(output_folder, exist_ok=True)

frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break  # already get the video
    
    frame_filename = os.path.join(output_folder, f"frame_{frame_idx:04d}.png")
    crop_frame = frame[100:900, 300:1400]  # [y1:y2, x1:x2]
    cv2.imwrite(frame_filename, crop_frame)
    
    frame_idx += 1

cap.release()
print(f"Saved {frame_idx} frames.")

Saved 1797 frames.


In [12]:
# Set the tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r"D:\tesseract\tesseract.exe"

# Set folders
input_folder = 'frames_Ex_1'          # Input image folder
output_csv = 'EX1.csv'          # Output CSV file

# Prepare to store timestamps
timestamps = []

# Get all image files
file_list = sorted([f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

# Updated regex: allow ":" or ";" and "." or ","
timestamp_pattern = re.compile(r'\d{2}[:;]\d{2}[:;]\d{2}[\.,]\d{3}')

def timestamp_to_seconds(timestamp_str):
    """Convert timestamp string to total seconds (float)."""
    h, m, s_ms = timestamp_str.split(":")
    s, ms = s_ms.split(".")
    total_seconds = int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000
    return round(total_seconds, 3)

for filename in file_list:
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path)

    if img is None:
        print(f"Warning: Cannot read {filename}")
        continue

    # Crop the timestamp region
    timestamp_area = img[10:100, 100:400]   # [y1:y2, x1:x2]
    gray = cv2.cvtColor(timestamp_area, cv2.COLOR_BGR2GRAY)

    # OCR to recognize the timestamp
    custom_config = r'--oem 3 --psm 7'
    timestamp_text = pytesseract.image_to_string(gray, config=custom_config).strip()

    print(f"{filename} -> {timestamp_text}")

    # Check if the recognized text matches the flexible timestamp format
    if not timestamp_pattern.fullmatch(timestamp_text):
        print(f"❌ Timestamp format error at {filename}: {timestamp_text}")
        print("⛔ Stopping the process due to invalid timestamp format.")
        break

    # Auto-correct recognized timestamp: replace ';' -> ':' and ',' -> '.'
    corrected_timestamp = timestamp_text.replace(';', ':').replace(',', '.')

    # Convert to total seconds
    total_seconds = timestamp_to_seconds(corrected_timestamp)

    # Save the total seconds directly
    timestamps.append([filename, total_seconds])

# Save all timestamps into a CSV file
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Filename", "Timestamp (seconds)"])
    writer.writerows(timestamps)

print("✅ Batch processing completed!")

frame_0000.png -> 10:18:20.669
frame_0001.png -> 10:18:20.720
frame_0002.png -> 10:18:20.768
frame_0003.png -> 10:18:20.828
frame_0004.png -> 10:18:20.877
frame_0005.png -> 10:18:20.925
frame_0006.png -> 10:18:20.925
frame_0007.png -> 10:18:20.981
frame_0008.png -> 10:18:20.981
frame_0009.png -> 10:18:21.027
frame_0010.png -> 10:18:21.027
frame_0011.png -> 10:18:21.074
frame_0012.png -> 10:18:21.126
frame_0013.png -> 10:18:21.180
frame_0014.png -> 10:18:21.226
frame_0015.png -> 10:18:21.226
frame_0016.png -> 10:18:21.283
frame_0017.png -> 10:18:21.283
frame_0018.png -> 10:18:21.343
frame_0019.png -> 10:18:21.383
frame_0020.png -> 10:18:21.383
frame_0021.png -> 10:18:21.433
frame_0022.png -> 10:18:21.433
frame_0023.png -> 10:18:21.483
frame_0024.png -> 10:18:21.483
frame_0025.png -> 10:18:21.539
frame_0026.png -> 10:18:21.583
frame_0027.png -> 10:18:21.583
frame_0028.png -> 10:18:21.641
frame_0029.png -> 10:18:21.686
frame_0030.png -> 10:18:21.743
frame_0031.png -> 10:18:21.743
frame_00

In [13]:
import pandas as pd
import numpy as np

ex1 = pd.read_csv('EX1.csv')
ex1_sensor = pd.read_csv('EX1_sensor.csv')

print(ex1.columns)
print(ex1_sensor.columns)

Index(['Filename', 'Timestamp (seconds)'], dtype='object')
Index(['Timestamp (s)', 'X (mm)', 'Y (mm)', 'Z (mm)', 'Roll (deg)',
       'Pitch (deg)', 'Yaw (deg)'],
      dtype='object')


In [14]:
ex1 = ex1.rename(columns={'Timestamp (seconds)': 'Timestamp (s)'})

# 要插值的欄位
interp_columns = ['X (mm)', 'Y (mm)', 'Z (mm)', 'Roll (deg)', 'Pitch (deg)', 'Yaw (deg)']

# 對每一個欄位進行線性內插
for col in interp_columns:
    ex1[col] = np.interp(
        ex1['Timestamp (s)'],           # timestamps corresponding to files
        ex1_sensor['Timestamp (s)'],     # timestamps of sensor
        ex1_sensor[col]              # data of sensor
    )

# output csv
ex1.to_csv('EX1_final_data.csv', index = False)

print("✅ EX1_final_data.csv is saved successfully!")

✅ EX1_final_data.csv is saved successfully!


In [15]:
ex1

,Filename,Timestamp (s),X (mm),Y (mm),Z (mm),Roll (deg),Pitch (deg),Yaw (deg)
0,frame_0000.png,37100.669,61.287754,-27.845274,-0.630689,6.679108,-1.802951,3.069303
1,frame_0001.png,37100.720,56.943810,-28.652644,-0.583040,6.633080,-1.892977,3.051654
2,frame_0002.png,37100.768,52.907731,-29.460925,-0.545920,6.866517,-1.781280,3.025068
3,frame_0003.png,37100.828,48.392813,-30.162169,-0.465169,6.996138,-1.731087,3.008335
4,frame_0004.png,37100.877,44.163926,-30.627892,-0.222308,6.618447,-1.841919,3.016846
...,...,...,...,...,...,...,...,...
1053,frame_1053.png,37135.639,-9.515590,23.572415,-0.885491,-0.373806,-0.377115,0.363502
1054,frame_1054.png,37135.639,-9.515590,23.572415,-0.885491,-0.373806,-0.377115,0.363502
1055,frame_1055.png,37135.734,-9.171573,22.470317,-0.987006,-0.417871,-0.239695,0.330523
1056,frame_1056.png,37135.734,-9.171573,22.470317,-0.987006,-0.417871,-0.239695,0.330523
